# Liver Model

## Import some things

In [1]:
import os
def return_sys_path():
    path = '.'
    for _ in range(5):
        if 'Deep_Learning' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path
def return_data_path():
    path = '.'
    for _ in range(5):
        if 'Data' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path

In [2]:
import os, sys
sys.path.append(return_sys_path())
from Deep_Learning.Base_Deeplearning_Code.Data_Generators.Generators import Data_Generator_Class, os
from Deep_Learning.Base_Deeplearning_Code.Keras_Utils.Keras_Utilities import np, dice_coef_3D
from Deep_Learning.Base_Deeplearning_Code.Data_Generators.Image_Processors import *
from Deep_Learning.Base_Deeplearning_Code.Callbacks.Visualizing_Model_Utils import TensorBoardImage
from tensorflow.python.keras.callbacks import ModelCheckpoint
# from Utils import ModelCheckpoint, model_path_maker
from Deep_Learning.Base_Deeplearning_Code.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image
import tensorflow.compat.v1 as tf
import tensorflow.python.keras.backend as K

W0217 17:43:22.398360 17780 deprecation.py:506] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
%matplotlib inline

In [4]:
%load_ext tensorboard

In [5]:
base = return_sys_path()
data_path = os.path.join(return_data_path(),'Data','Niftii_Arrays')
train_path = [os.path.join(data_path,'Train')]
validation_path = os.path.join(data_path,'Validation')
test_path = os.path.join(data_path,'Test')
model_path = os.path.join(base,'Models')
if not os.path.exists(model_path):
    os.makedirs(model_path)

## We now need some image processors...

#### We will ensure that the images are 256 x 256 (downsampled for speed), normalize them with a mean of 78 and std of 29, add random noise, threshold, and turn into 2 classes

In [6]:
image_size = 128
image_processors_train = [Ensure_Image_Proportions(image_size,image_size),Repeat_Channel(repeats=3),
                          Normalize_Images(mean_val=78,std_val=29),
                          Threshold_Images(lower_bound=-3.55,upper_bound=3.55, post_load=False),
                          Annotations_To_Categorical(2)]
#Add_Noise_To_Images(variation=np.round(np.arange(start=0, stop=0.3, step=0.1),2)),
image_processors_test = [Ensure_Image_Proportions(image_size,image_size),Normalize_Images(mean_val=78,std_val=29),
                         Repeat_Channel(repeats=3), Threshold_Images(lower_bound=-3.55,upper_bound=3.55),
                         Annotations_To_Categorical(2)]

In [7]:
batch_size = 5
train_generator = Data_Generator_Class(by_patient=False,shuffle=True,data_paths=train_path,batch_size=batch_size,
                                       image_processors=image_processors_train)
validation_generator = Data_Generator_Class(by_patient=True,shuffle=True,data_paths=validation_path,batch_size=30,
                                            image_processors=image_processors_test, by_patient_2D=True)

### Lets visualize one of the examples! With batch_size of 5 and shuffle on, it will be 5 random 2D slices

In [8]:
x,y = train_generator.__getitem__(0)

In [9]:
plot_Image_Scroll_Bar_Image(x)

interactive(children=(IntSlider(value=0, description='Z', max=4), Text(value='2D', description='view'), Output…

### Alright, lets make our model!

In [ ]:
from Deep_Learning.Easy_VGG16_UNet.Keras_Fine_Tune_VGG_16_Liver import VGG_16
from Deep_Learning.Base_Deeplearning_Code.Visualizing_Model.Visualing_Model import visualization_model_class
from tensorflow.python.keras.optimizers import Adam

### This is just a click and play, it builds the VGG16 architecture for you with pre-trained weights

![VGG16_Unet.png](./Deep_Learning/Easy_VGG16_UNet/VGG16_UNet.png)

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [64]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model
model_path = os.path.join(return_sys_path(),'Models')

## These are some tools for visualizing the model

In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

### Lets look at the activations of block1_conv1, the activation, and output

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','block1_conv1_activation','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x[0,...][None,...])

In [ ]:
Visualizing_Class.plot_activations()

In [ ]:
new_model.compile(Adam(lr=5e-5),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

## Freezing pre-trained layers

In [ ]:
def freeze_until_name(model,name):
    set_trainable = False
    for layer in model.layers:
        if layer.name == name:
            set_trainable = True
        layer.trainable = set_trainable
    return model
new_model = freeze_until_name(new_model,'Upsampling0_UNet')

## Checkpoint and run

A checkpoint is a way of assessing the model and determining if we should save it

In [ ]:
model_name = 'VGG_16_Model'
other_aspects = [model_name,'Upsampling0_UNet_Unfrozen'] # Just a list of defining things
model_path_out = os.path.join(model_path,'VGG_16_frozen')

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator)
callbacks = [checkpoint, tensorboard]

In [ ]:
%tensorboard --logdir {"../Models"}

In [ ]:
new_model.fit_generator(train_generator,epochs=5, workers=20, max_queue_size=50,
                        validation_data=validation_generator,callbacks=callbacks,
                        steps_per_epoch=200, image_frequency=1)

In [ ]:
x,y = validation_generator.__getitem__(0)

In [ ]:
pred = new_model.predict(x)

In [ ]:
pred[pred<0.5] = 0
pred[pred>0] = 1

In [ ]:
plot_scroll_Image(pred[...,1])

# Now lets make our own architecture

### First, lets import some necessary functions

In [10]:
from Deep_Learning.Base_Deeplearning_Code.Models.Keras_3D_Models import my_3D_UNet
from Deep_Learning.Base_Deeplearning_Code.Cyclical_Learning_Rate.clr_callback import CyclicLR
from tensorflow.python.keras.callbacks import ModelCheckpoint
from functools import partial
from tensorflow.python.keras.optimizers import Adam

### Define our convolution and strided blocks, strided is used for downsampling

In [11]:
activation = 'relu'
kernel = (3,3)
pool_size = (2,2)
#{'channels': x, 'kernel': (3, 3), 'strides': (1, 1),'activation':activation}
conv_block = lambda x: {'channels': x}
strided_block = lambda x: {'channels': x, 'strides': (2, 2)}
pooling_downsampling = {'pooling': {'pooling_type': 'Max',
                                    'pool_size': (2, 2), 'direction': 'Down'}}
pooling_upsampling = {'pooling': {'pool_size': (2, 2), 'direction': 'Up'}}

### Our architecture will have 2 main parts in each layer, an 'Encoding' and a 'Decoding' side, 'Encoding' goes down, and 'Decoding' goes up

![Encoding and Decoding.png](./Deep_Learning/Encoding_and_Decoding.png)

### We will now create our layer dictionary, this tells our UNet what to look like

### If Pooling is left {} it will perform maxpooling and upsampling with pooling()

In [14]:
layers_dict = {}
filters = 16
layers_dict['Layer_0'] = {'Encoding':[conv_block(filters),conv_block(filters)],
                          'Decoding':[conv_block(filters),conv_block(filters)],
                          'Pooling':{'Encoding':pooling_downsampling,
                                     'Decoding':pooling_upsampling
                                    }}
filters = 32
layers_dict['Base'] = {'Encoding':[conv_block(filters), conv_block(filters),conv_block(filters)]}

In [15]:
layers_dict

{'Layer_0': {'Encoding': [{'channels': 16}, {'channels': 16}],
  'Decoding': [{'channels': 16}, {'channels': 16}],
  'Pooling': {}},
 'Base': {'Encoding': [{'channels': 32}, {'channels': 32}, {'channels': 32}]}}

In [16]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
new_model = my_3D_UNet(kernel=kernel,layers_dict=layers_dict, pool_size=pool_size,activation=activation,is_2D=True,
                       input_size=3, image_size=image_size).created_model

Layer_0
Base
Layer_0


### Set a learning rate and loss metric, also add any metrics you want to track

In [17]:
min_lr = 5e-6
max_lr = 8e-4
new_model.compile(Adam(lr=min_lr),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

### Name your model and define other things! Send a list of strings and it will make a folder path

In [18]:
model_name = 'My_New_Model'
other_aspects = [model_name,'{}_Layers'.format(3),'{}_Conv_Blocks'.format(2),
                 '{}_Filters'.format(16),'{}_MinLR_{}_MaxLR'.format(min_lr,max_lr)] # Just a list of defining things
model_path_out = model_path_maker(model_path,other_aspects)

NameError: name 'model_path_maker' is not defined

### A few more parameters...

In [19]:
steps_per_epoch = len(train_generator)//3
step_size_factor = 2

### This is a checkpoint to save the model if it has the highest dice, also to add images

#### We will specify that we want to watch the validation dice, and save the one with the highest value

In [29]:
monitor = 'val_dice_coef_3D'
mode = 'max'
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor=monitor, verbose=1, save_best_only=True,
                             save_weights_only=False, save_freq='epoch', mode=mode)

#### Next, our tensorboard output will add prediction images

In [23]:
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=os.path.join('.','Models'), batch_size=1, num_images=1,update_freq='epoch', 
                               data_generator=validation_generator, image_frequency=3)


W0217 17:45:50.838107 17780 Visualizing_Model_Utils.py:520] `batch_size` is no longer needed in the `TensorBoard` Callback and will be ignored in TensorFlow 2.0.


#### CyclicLR will allow us to change the learning rate of the model as it runs, and Add_LR_To_Tensorboard will let us view it later

In [ ]:
cyclic_lrate = CyclicLR(base_lr=min_lr, max_lr=max_lr, step_size=steps_per_epoch * step_size_factor, mode='triangular2')
add_lr_to_tensorboard = Add_LR_To_Tensorboard()

### Combine all callbacks

In [ ]:
callbacks = [cyclic_lrate, add_lr_to_tensorboard, tensorboard, checkpoint]

In [24]:
new_model.fit_generator(train_generator,epochs=10, workers=10, max_queue_size=200, validation_data=validation_generator,
                       callbacks=callbacks, steps_per_epoch=steps_per_epoch)

Epoch 1/10


W0217 17:46:03.330332 17780 deprecation.py:323] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


279/279 [==============================] - 124s 444ms/step - loss: 0.6445 - acc: 0.6844 - dice_coef_3D: 0.1133 - val_loss: 0.5456 - val_acc: 0.9282 - val_dice_coef_3D: 0.1011
Epoch 2/10
279/279 [==============================] - 91s 327ms/step - loss: 0.4490 - acc: 0.9249 - dice_coef_3D: 0.1297 - val_loss: 0.3606 - val_acc: 0.9016 - val_dice_coef_3D: 0.1957
Epoch 3/10
279/279 [==============================] - 82s 295ms/step - loss: 0.2384 - acc: 0.9347 - dice_coef_3D: 0.1845 - val_loss: 0.1918 - val_acc: 0.9103 - val_dice_coef_3D: 0.2922
Epoch 4/10
279/279 [==============================] - 77s 277ms/step - loss: 0.1460 - acc: 0.9384 - dice_coef_3D: 0.2452 - val_loss: 0.1757 - val_acc: 0.8943 - val_dice_coef_3D: 0.3434
Epoch 5/10
279/279 [==============================] - 76s 273ms/step - loss: 0.1266 - acc: 0.9365 - dice_coef_3D: 0.2937 - val_loss: 0.1456 - val_acc: 0.9151 - val_dice_coef_3D: 0.3360
Epoch 6/10
279/279 [==============================] - 78s 278ms/step - loss: 0.1166 -

In [ ]:
%tensorboard --logdir {"./Models"}